---
# Minicurso: **CURSO DE APLICAÇÕES DE SATÉLITE PARA NOWCASTING**

---

**OBJETIVO:** A parte prática do curso tem como objetivo ensinar a utilização dos dados
e produtos de satélite para monitoramento em tempo real e nowcasting
(previsão dentro de um período de 0-6 horas).

---

**EMENTA**: Será abordado os seguintes tópicos

1. Download dos dados do sensor ABI e GLM do GOES-16
2. Transformação da projeção satélite para retangular
3. Plotar imagens em escala de cinza
4. Plotar imagens em escala colorida
5. Gerar imagem do IR combinado com o total de flashes
6. Gerar imagem do IR combinado com a densidade de flashes
7. Rastreamento manual de tempestade e evolução temporal da temperatura de bilho do IR e flashes
---

**DADOS:**
- A parte prática do curso utilizará como informações de entrada os dados do sensor Advanced Baseline Imager (ABI) e Geostationary Lightning Mapper (GLM) abordo do satélite Geostationary Operational Environmental Satellite - 16. O período de dados é referente aos desastres naturais que ocorreram no estado do Rio Grande do Sul entre 26 de abril e 5 de maio de 2024. O INMET registrou para o mês de Maio um total de 617,1 mm, o que indicou uma chuva acima da média no valor de `480,5 mm`. Em adição, a estação pluviômétrica do CEMADEN localizada em Santa Maria (RS) registrou `236 mm` no dia 30 de abril. Os desastres afetaram 2.390.556 pessoas e provocaram a morte de 172 pessoas.

- Maiores detalhes sobre o evento pode ser encontrado na Nota Técnica emitida pelo [INMET-Eventos Extremos de Março de 2024 no Brasil](https://github.com/evmpython/minicurso_nowcasting_CPAM2024/blob/main/docs/EventosExtremos-Brasil-Maio-2024.pdf) e [CEMADEN - Boletim de Impactos de Extremos de Origem Hidro-Geo-Climático em Atividades Estratégicas para o Brasil – 14/05/2024 ANO 07 Nº 66](https://www.gov.br/cemaden/pt-br/assuntos/monitoramento/boletim-de-impactos/copy4_of_boletim-de-impactos-de-extremos-de-origem-hidro-geo-climatico-em-atividades-estrategicas-para-o-brasil-2013-17-01-2024-ano-07-no-62).


---

**PROCEDIMENTO REALIZADO:** Os seguintes procedimentos serão realizados nesse código:
1. 1° Passo: Verificações preliminares
2. 2° Passo: Instalando bibliotecas
3. 3° Passo: Download de arquivos auxiliares
4. **Script 01** - Baixando Dados da Nuvem - AWS
5. **Script 02** - Projeção Retangular e Imagem em Níveis de Cinza
6. **Script 03** - Projeção Retangular com Imagem Colorida
7. **Script 04** - Imagem IR + Total de Flashes do GLM
8. **Script 05** - Imagem IR + Densidade de Flashes do GLM
9. **Script 06** - Evolução Temporal da Temperatura de Brilho do IR e Flashes
---

**OBSERVAÇÕES IMPORTANTES**:
1. Este código foi desenvolvido para ser processado no [Google Colaboratory](https://colab.research.google.com/).
2. Os dados estão disponíveis no [github do curso](https://github.com/evmpython/minicurso_nowcasting_CPAM2024).

---

**Equipe:**

Palestrantes/Tutores:
 - Diego Souza - INPE: diego.souza@inpe.br / https://github.com/diegormsouza

 - Thiago Souza Biscaro - INPE: thiago.biscaro@inpe.br / https://github.com/tsbiscaro

 - Douglas Uba - INPE: douglas.uba@inpe.br / https://github.com/uba

 - Enrique Vieira Mattos - UNIFEI: enrique@unifei.edu.br / https://github.com/evmpython

 - Vito Galligani - CONICET/UBA/Argentina: vito.galligani@gmail.com

Colaboradores:
 - Flávio Augusto - UNIFEI: augustoflaviobob@gmail.com
---

# **1° Passo:**  Verificações Preliminares

Neste passo verificamos a configuração da máquina e a versão Python instalada.

In [ ]:
# Verificando configuração da máquina
!cat /etc/issue
!uname -a
print('\n')

# Verificando a memória
!grep MemTotal /proc/meminfo
print('\n')

# Verificando o HD
!df -h
print('\n')

# Verificando qual o diretório da instalação padrão do Python
!which python
print('\n')

# Verificando qual a versão instalada do Python
!python --version

# **2° Passo:** Instalando as Bibliotecas Necessárias

Neste passo instalamos as bibliotecas necessárias para a execução dos scripts. Basicamente, as bibliotecas terão a seguinte finalidade:

*   **NetCDF4:** Ler os dados de arquivos no formato NetCDF4
*   **Cartopy:** Adicionar mapas aos plots
*   **Boto3:** Download de dados GOES-16 da Amazon
*   **GDAL:** Reprojeção de imagens GOES-16
*   **SALEM:** Mascaramento dos dados

In [ ]:
# Instalando as bibliotecas numpy, netcdf4 e boto3
!pip install "numpy<2.0" netcdf4 cartopy boto3

# Instalando a biblioteca Salem
!pip install -q rasterio pyproj geopandas salem descartes

# Instalando / atualizando a biblioteca GDAL
!apt-add-repository -y ppa:ubuntugis/ubuntugis-unstable
!add-apt-repository -y ppa:ubuntugis/ppa
!apt-get install gdal-bin
!pip install gdal

# **3° Passo:** Download de Arquivos Auxiliares

Neste passo vamos baixar alguns arquivos auxiliares necessários para parte dos scripts que serão demonstrados no curso:

*   **utilities.py:** Script com algumas funções para processamento de dados de satélite
*   **IR4AVHRR6.cpt:** Paleta de cores para canais infravermelhos do GOES-16

In [ ]:
# Download do script utilities.py
!wget -c https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/utils/utilities.py
print('\n')

# Download da arquivo CPT exemplo
!wget -c https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/utils/IR4AVHRR6.cpt
print('\n')

#**Script 01** - Baixando Dados da Nuvem - AWS

In [ ]:
#========================================================================================================================#
#                                            REQUIRED MODULES
#========================================================================================================================#
from netCDF4 import Dataset              # Read / Write NetCDF4 files
import matplotlib.pyplot as plt          # Plotting library
from datetime import datetime            # Basic Dates and time types
import cartopy, cartopy.crs as ccrs      # Plot maps
import os                                # Miscellaneous operating system interfaces
from utilities import download_CMI       # Our own utilities

#========================================================================================================================#
#                                          INPUT AND OUTPUT DIRECTORIES
#========================================================================================================================#
input = "/content/samples"; os.makedirs(input, exist_ok=True)
output = "/content/output"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                              DOWNLOAD THE FILE
#========================================================================================================================#
# Datetime to process
yyyymmddhhmn = '202404301300'

# Initial time and date
yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

# Channel
band = '13'

# Download the file
file_name = download_CMI(yyyymmddhhmn, band, input)

#========================================================================================================================#
#                                           READ THE REPROJECTED FILE
#========================================================================================================================#
# Open the GOES-R image
file = Dataset(f'{input}/{file_name}.nc')

# Get the pixel values
data = file.variables['CMI'][:] - 273.15

#========================================================================================================================#
#                                                 PLOT THE FIGURE
#========================================================================================================================#
# Choose the plot size (width x height, in inches)
plt.figure(figsize=(10,10))

# Use the Geostationary projection in cartopy
ax = plt.axes(projection=ccrs.Geostationary(central_longitude=-75.0, satellite_height=35786023.0))
img_extent = (-5434894.67527, 5434894.67527, -5434894.67527, 5434894.67527)

# Add coastlines, borders and gridlines
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
ax.gridlines(color='white', alpha=0.5, linestyle='--', linewidth=0.5)

# Define the color scale based on the channel
colormap = "gray_r" # White to black for IR channels

# Plot the image
img = ax.imshow(data, origin='upper', vmin=-80, vmax=40, extent=img_extent, cmap=colormap)

# Add a colorbar
plt.colorbar(img, label='Brightness Temperatures (°C)', extend='both', orientation='vertical', pad=0.05, fraction=0.05)

# read the time/date from the NetCDF file metadata as a string
date_string = file.getncattr('time_coverage_start')
date_format = '%Y-%m-%dT%H:%M:%S.%fZ'
date_obj = datetime.strptime(date_string, date_format)
date = date_obj.strftime('%Y-%m-%d %H:%M UTC')

# Add a title
plt.title(f'GOES-16 Band 13 (10.3 µm)\n{date}', fontweight='bold', fontsize=10, loc='left')
plt.title('Full Disk', fontsize=10, loc='right')

# Save the image
plt.savefig(f'{output}/script_1_{yyyy}{mm}{dd}_{hh}{mn}.png')

# Show the image
plt.show()

#**Script 02** - Projeção Retangular e Imagem em Níveis de Cinza

In [ ]:
#========================================================================================================================#
#                                            REQUIRED MODULES
#========================================================================================================================#
from netCDF4 import Dataset                     # Read / Write NetCDF4 files
import matplotlib.pyplot as plt                 # Plotting library
from datetime import datetime                   # Basic Dates and time types
import cartopy, cartopy.crs as ccrs             # Plot maps
import os                                       # Miscellaneous operating system interfaces
from osgeo import osr                           # Python bindings for GDAL
from osgeo import gdal                          # Python bindings for GDAL
import numpy as np                              # Scientific computing with Python
from utilities import download_CMI              # Our function for download

#========================================================================================================================#
#                                          INPUT AND OUTPUT DIRECTORIES
#========================================================================================================================#
input = "/content/samples"; os.makedirs(input, exist_ok=True)
output = "/content/output"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                              DOWNLOAD THE FILE
#========================================================================================================================#
# Datetime to process
yyyymmddhhmn = '202404301300'

# Initial time and date
yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

# Channel
band = '13'

# Variable
var = 'CMI'

# Desired extent
extent = [-60.00, -40.00, -35.00, -25.00] # Min lon, Max lon, Min lat, Max lat

# Download the file
file_name = download_CMI(yyyymmddhhmn, band, input)

#========================================================================================================================#
#                                               REPROJECT THE DATA
#========================================================================================================================#
# Open the file
img = gdal.Open(f'NETCDF:{input}/{file_name}.nc:' + var)

# Read the header metadata
metadata = img.GetMetadata()
scale = float(metadata.get(var + '#scale_factor'))
offset = float(metadata.get(var + '#add_offset'))
undef = float(metadata.get(var + '#_FillValue'))
dtime = metadata.get('NC_GLOBAL#time_coverage_start')

# Load the data
ds = img.ReadAsArray(0, 0, img.RasterXSize, img.RasterYSize).astype(float)

# Apply the scale, offset and convert to celsius
ds = (ds * scale + offset) - 273.15

# Read the original file projection and configure the output projection
source_prj = osr.SpatialReference()
source_prj.ImportFromProj4(img.GetProjectionRef())

target_prj = osr.SpatialReference()
target_prj.ImportFromProj4("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

# Reproject the data
GeoT = img.GetGeoTransform()
driver = gdal.GetDriverByName('MEM')
raw = driver.Create('raw', ds.shape[0], ds.shape[1], 1, gdal.GDT_Float32)
raw.SetGeoTransform(GeoT)
raw.GetRasterBand(1).WriteArray(ds)

# Define the parameters of the output file
options = gdal.WarpOptions(format = 'netCDF',
          srcSRS = source_prj,
          dstSRS = target_prj,
          outputBounds = (extent[0], extent[3], extent[2], extent[1]),
          outputBoundsSRS = target_prj,
          outputType = gdal.GDT_Float32,
          srcNodata = undef,
          dstNodata = 'nan',
          xRes = 0.02,
          yRes = 0.02,
          resampleAlg = gdal.GRA_NearestNeighbour)

print(options)

# Write the reprojected file on disk
gdal.Warp(f'{output}/{file_name}_ret.nc', raw, options=options)

#========================================================================================================================#
#                                               READ THE REPROJECTED FILE
#========================================================================================================================#
# Open the reprojected GOES-R image
file = Dataset(f'{output}/{file_name}_ret.nc')

# Get the pixel values
data = file.variables['Band1'][:]

#========================================================================================================================#
#                                                 PLOT THE FIGURE
#========================================================================================================================#
# Choose the plot size (width x height, in inches)
plt.figure(figsize=(10,10))

# Use the Geostationary projection in cartopy
ax = plt.axes(projection=ccrs.PlateCarree())

# Define the image extent
img_extent = [extent[0], extent[2], extent[1], extent[3]]

# Define the color scale based on the channel
colormap = "gray_r" # White to black for IR channels

# Plot the image
img = ax.imshow(data, origin='upper', vmin=-80, vmax=40, extent=img_extent, cmap=colormap)

# Add coastlines, borders and gridlines
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Plot states
shapefile = list(shpreader.Reader('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

# Add a colorbar
plt.colorbar(img, label='Brightness Temperatures (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)

# read the time/date from the NetCDF file metadata as a string
date_format = '%Y-%m-%dT%H:%M:%S.%fZ'
date_obj = datetime.strptime(dtime, date_format)
date = date_obj.strftime('%Y-%m-%d %H:%M UTC')

# Add a title
plt.title(f'GOES-16 Band 13 (10.3 µm)\n{date}', fontweight='bold', fontsize=10, loc='left')
plt.title('Reg.: ' + str(extent) , fontsize=10, loc='right')

# Save the image
plt.savefig(f'{output}/script_2_{yyyy}{mm}{dd}_{hh}{mn}.png', bbox_inches='tight', dpi=300)

# Show the image
plt.show()

#**Script 03** - Projeção Retangular com Imagem Colorida

In [ ]:
#========================================================================================================================#
#                                            REQUIRED MODULES
#========================================================================================================================#
from netCDF4 import Dataset                     # Read / Write NetCDF4 files
import matplotlib.pyplot as plt                 # Plotting library
from datetime import datetime                   # Basic Dates and time types
import cartopy, cartopy.crs as ccrs             # Plot maps
import cartopy.io.shapereader as shpreader      # Read shapefiles
import os                                       # Miscellaneous operating system interfaces
from osgeo import osr                           # Python bindings for GDAL
from osgeo import gdal                          # Python bindings for GDAL
import numpy as np                              # Scientific computing with Python
from matplotlib import cm                       # Colormap handling utilities
from utilities import loadCPT                   # Import the CPT convert function
from utilities import download_CMI              # Our function for download
import xarray as xr                             # Work with multidimensional arrays

#========================================================================================================================#
#                                        INPUT AND OUTPUT DIRECTORIES
#========================================================================================================================#
input = "/content/samples"; os.makedirs(input, exist_ok=True)
output = "/content/output"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                              DOWNLOAD THE FILE
#========================================================================================================================#
# Datetime to process
yyyymmddhhmn = '202404301300'

# Initial time and date
yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

# Channel
band = '13'

# Variable
var = 'CMI'

# Desired extent
extent = [-60.00, -40.00, -35.00, -25.00] # Min lon, Max lon, Min lat, Max lat

# Download the file
file_name = download_CMI(yyyymmddhhmn, band, input)

#========================================================================================================================#
#                                               REPROJECT THE DATA
#========================================================================================================================#
# Open the file
img = gdal.Open(f'NETCDF:{input}/{file_name}.nc:' + var)

# Read the header metadata
metadata = img.GetMetadata()
scale = float(metadata.get(var + '#scale_factor'))
offset = float(metadata.get(var + '#add_offset'))
undef = float(metadata.get(var + '#_FillValue'))
dtime = metadata.get('NC_GLOBAL#time_coverage_start')

# Load the data
ds = img.ReadAsArray(0, 0, img.RasterXSize, img.RasterYSize).astype(float)

# Apply the scale, offset and convert to celsius
ds = (ds * scale + offset) - 273.15

# Read the original file projection and configure the output projection
source_prj = osr.SpatialReference()
source_prj.ImportFromProj4(img.GetProjectionRef())

target_prj = osr.SpatialReference()
target_prj.ImportFromProj4("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

# Reproject the data
GeoT = img.GetGeoTransform()
driver = gdal.GetDriverByName('MEM')
raw = driver.Create('raw', ds.shape[0], ds.shape[1], 1, gdal.GDT_Float32)
raw.SetGeoTransform(GeoT)
raw.GetRasterBand(1).WriteArray(ds)

# Define the parameters of the output file
options = gdal.WarpOptions(format = 'netCDF',
                           srcSRS = source_prj,
                           dstSRS = target_prj,
                           outputBounds = (extent[0], extent[3], extent[2], extent[1]),
                           outputBoundsSRS = target_prj,
                           outputType = gdal.GDT_Float32,
                           srcNodata = undef,
                           dstNodata = 'nan',
                           xRes = 0.02,
                           yRes = 0.02,
                           resampleAlg = gdal.GRA_NearestNeighbour)

# Write the reprojected file on disk
gdal.Warp(f'{output}/{file_name}_ret.nc', raw, options=options)

#========================================================================================================================#
#                                               READ THE REPROJECTED FILE
#========================================================================================================================#
data = xr.open_dataset(f'{output}/{file_name}_ret.nc')

#========================================================================================================================#
#                                                 CREATE THE FIGURE
#========================================================================================================================#
# Choose the plot size (width x height, in inches)
plt.figure(figsize=(10,10))

# Use the Geostationary projection in cartopy
ax = plt.axes(projection=ccrs.PlateCarree())

# Define the image extent
img_extent = [extent[0], extent[2], extent[1], extent[3]]

# Converts a CPT file to be used in Python
cpt = loadCPT('IR4AVHRR6.cpt')
cmap = cm.colors.LinearSegmentedColormap('cpt', cpt)

# Plot the image
img = ax.imshow(data['Band1'], origin='upper', vmin=-103.0, vmax=84, extent=img_extent, cmap=cmap, alpha=1.0)

# Add coastlines, borders and gridlines
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Plot states
shapefile = list(shpreader.Reader('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

# Add a colorbar
plt.colorbar(img, label='Brightness Temperatures (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)

# Add a title
plt.title(f'GOES-16 Band 13 (10.3 µm)\n{yyyy}-{mm}-{dd} {hh}:{mn} UTC', fontweight='bold', fontsize=10, loc='left')
plt.title('Reg.: ' + str(extent) , fontsize=10, loc='right')

# Save the image
plt.savefig(f'{output}/script_3_{yyyy}{mm}{dd}_{hh}{mn}.png', bbox_inches='tight', dpi=300)

# Show the image
plt.show()

#**Script 04** - Imagem IR + Total de Flashes do GLM

In [ ]:
#========================================================================================================================#
#                                            REQUIRED MODULES
#========================================================================================================================#
from netCDF4 import Dataset                                 # Read / Write NetCDF4 files
import matplotlib.pyplot as plt                             # Plotting library
import cartopy, cartopy.crs as ccrs                         # Plot maps
from datetime import timedelta, date, datetime              # Basic Dates and time types
import os                                                   # Miscellaneous operating system interfaces
from osgeo import gdal                                      # Python bindings for GDAL
import numpy as np                                          # Scientific computing with Python
from matplotlib import cm                                   # Colormap handling utilities
from utilities import download_CMI, download_GLM, loadCPT   # Import the CPT convert function   # Our function for download
from utilities import reproject                             # Our function for reproject
import cartopy.io.shapereader as shpreader                  # Read shapefiles
import xarray as xr                                         # Work with multidimensional arrays
import pandas as pd                                         # Work with dataframes
gdal.PushErrorHandler('CPLQuietErrorHandler')               # Ignore GDAL warnings

#========================================================================================================================#
#                                        INPUT AND OUTPUT DIRECTORIES
#========================================================================================================================#
input = "/content/samples"; os.makedirs(input, exist_ok=True)
output = "/content/output"; os.makedirs(output, exist_ok=True)

#========================================================================================================================#
#                                              DOWNLOAD THE FILE
#========================================================================================================================#
# Datetime to process
yyyymmddhhmn = '202404301300'

# Initial time and date
yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

# Channel
band = '13'

# Variable
var = 'CMI'

# Desired extent
extent = [-60.00, -40.00, -35.00, -25.00] # Min lon, Max lon, Min lat, Max lat

# Download the file
file_ir = download_CMI(yyyymmddhhmn, band, input)

#========================================================================================================================#
#                                               REPROJECT THE DATA
#========================================================================================================================#
# Open the file
img = gdal.Open(f'NETCDF:{input}/{file_ir}.nc:' + var)

# Data Quality Flag (DQF)
dqf = gdal.Open(f'NETCDF:{input}/{file_ir}.nc:DQF')

# Read the header metadata
metadata = img.GetMetadata()
scale = float(metadata.get(var + '#scale_factor'))
offset = float(metadata.get(var + '#add_offset'))
undef = float(metadata.get(var + '#_FillValue'))
dtime = metadata.get('NC_GLOBAL#time_coverage_start')

# Load the data
ds_cmi = img.ReadAsArray(0, 0, img.RasterXSize, img.RasterYSize).astype(float)
ds_dqf = dqf.ReadAsArray(0, 0, dqf.RasterXSize, dqf.RasterYSize).astype(float)

# Apply the scale, offset and convert to celsius
ds_cmi = (ds_cmi * scale + offset) - 273.15

# Apply NaN's where the quality flag is greater than 1
ds_cmi[ds_dqf > 1] = np.nan

# Reproject the file
filename_ret = f'{output}/IR_{yyyymmddhhmn}.nc'
reproject(filename_ret, img, ds_cmi, extent, undef)

#========================================================================================================================#
#                                               READ THE REPROJECTED FILE
#========================================================================================================================#
dataset_ir = xr.open_dataset(filename_ret)

#========================================================================================================================#
#                                               GET GLM DATA
#========================================================================================================================#
# Initialize arrays for latitude, longitude, and event energy
lats_flash = np.array([])
lons_flash = np.array([])

date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))
date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(minutes=10))

# GLM accumulation loop
while (date_ini <= date_end):

    # Date structure
    yyyymmddhhmnss = datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # Download the file
    file_glm = download_GLM(yyyymmddhhmnss, input)

    # Read the file
    glm_20s = Dataset(f'{input}/{file_glm}.nc')

    # Append lats / longs
    lats_flash = np.append(lats_flash, glm_20s.variables['flash_lat'][:])
    lons_flash = np.append(lons_flash, glm_20s.variables['flash_lon'][:])

    # Increment the date_ini
    date_ini = str(datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))

# Select the flashes inside the region
# put the flashes into dataframe
data_flash = {'lat': lats_flash, 'lon': lons_flash}
df = pd.DataFrame(data_flash)

# select the flashes from region of interest
df_flash_filtered = df[ (df['lon'] > extent[0]) & (df['lon'] < extent[1]) & (df['lat'] > extent[2]) & (df['lat'] < extent[3])]

# transform from dataframe to array
lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

#========================================================================================================================#
#                                                 CREATE THE FIGURE
#========================================================================================================================#
# Choose the plot size (width x height, in inches)
plt.figure(figsize=(14,11))

# Use the Geostationary projection in cartopy
ax = plt.axes(projection=ccrs.PlateCarree())

# Define the data extent
img_extent = [extent[0], extent[2], extent[1], extent[3]]

# Converts a CPT file to be used in Python
cpt = loadCPT('IR4AVHRR6.cpt')
cmap = cm.colors.LinearSegmentedColormap('cpt', cpt)

# Plot the image
img = ax.imshow(dataset_ir['Band1'], origin='upper', vmin=-103.0, vmax=84, extent=img_extent, cmap=cmap, alpha=1.0)

# Plot the GLM Data
glm = plt.scatter(lons_flash_filtered, lats_flash_filtered, transform=ccrs.PlateCarree(), marker='x', s=20, color='blue', alpha=0.5, zorder=2,
                  label=f'Flashes={str(len(lats_flash_filtered)).zfill(4)}')

# Add coastlines, borders and gridlines
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Plot states
shapefile = list(shpreader.Reader('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

# Add a colorbar
plt.colorbar(img, label='Brightness Temperatures (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)

# Add legend
ax.legend(loc='lower right', ncols=1, frameon=True)

# Add a title
plt.title(f'GOES-16 Band 13 (10.3 µm)\n{yyyy}-{mm}-{dd} {hh}:{mn} UTC', fontweight='bold', fontsize=10, loc='left')
plt.title('Reg.: ' + str(extent) , fontsize=10, loc='right')

# Save the image
plt.savefig(f'{output}/script_4_{yyyy}{mm}{dd}_{hh}{mn}.png', bbox_inches='tight', dpi=300)

# Show the image
plt.show()

#**Script 05** - Imagem IR + Densidade de Flashes do GLM

In [ ]:
#========================================================================================================================#
#                                            REQUIRED MODULES
#========================================================================================================================#
from netCDF4 import Dataset                         # Read / Write NetCDF4 files
import matplotlib.pyplot as plt                     # Plotting library
import cartopy, cartopy.crs as ccrs                 # Plot maps
from datetime import timedelta, date, datetime      # Basic Dates and time types
import cartopy, cartopy.crs as ccrs                 # Plot maps
import os                                           # Miscellaneous operating system interfaces
from osgeo import gdal                              # Python bindings for GDAL
import numpy as np                                  # Scientific computing with Python
from matplotlib import cm                           # Colormap handling utilities
from utilities import download_CMI, download_GLM    # Our function for download
from utilities import reproject                     # Our function for reproject
gdal.PushErrorHandler('CPLQuietErrorHandler')       # Ignore GDAL warnings

#========================================================================================================================#
#                                              DOWNLOAD THE FILE
#========================================================================================================================#
# Datetime to process
yyyymmddhhmn = '202404301300'

# Initial time and date
yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

# Channel
band = '13'

# Variable
var = 'CMI'

# Desired extent
extent = [-60.00, -40.00, -35.00, -25.00] # Min lon, Max lon, Min lat, Max lat

# Download the file
file_ir = download_CMI(yyyymmddhhmn, band, input)

#========================================================================================================================#
#                                               REPROJECT THE DATA
#========================================================================================================================#
# Open the file
img = gdal.Open(f'NETCDF:{input}/{file_ir}.nc:' + var)

# Data Quality Flag (DQF)
dqf = gdal.Open(f'NETCDF:{input}/{file_ir}.nc:DQF')

# Read the header metadata
metadata = img.GetMetadata()
scale = float(metadata.get(var + '#scale_factor'))
offset = float(metadata.get(var + '#add_offset'))
undef = float(metadata.get(var + '#_FillValue'))
dtime = metadata.get('NC_GLOBAL#time_coverage_start')

# Load the data
ds_cmi = img.ReadAsArray(0, 0, img.RasterXSize, img.RasterYSize).astype(float)
ds_dqf = dqf.ReadAsArray(0, 0, dqf.RasterXSize, dqf.RasterYSize).astype(float)

# Apply the scale, offset and convert to celsius
ds_cmi = (ds_cmi * scale + offset) - 273.15

# Apply NaN's where the quality flag is greater than 1
ds_cmi[ds_dqf > 1] = np.nan

# Reproject the file
filename_ret = f'{output}/IR_{yyyymmddhhmn}.nc'
reproject(filename_ret, img, ds_cmi, extent, undef)

#========================================================================================================================#
#                                               READ THE REPROJECTED FILE
#========================================================================================================================#
dataset_ir = xr.open_dataset(filename_ret)

#========================================================================================================================#
#                                               GET GLM DATA
#========================================================================================================================#
# Initialize arrays for latitude, longitude, and event energy
lats = np.array([])
lons = np.array([])

date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))
date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(minutes=10))

# GLM accumulation loop
while (date_ini <= date_end):

    # Date structure
    yyyymmddhhmnss = datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # Download the file
    file_glm = download_GLM(yyyymmddhhmnss, input)

    # Read the file
    glm_20s = Dataset(f'{input}/{file_glm}.nc')

    # Append lats / longs
    lats_flash = np.append(lats_flash, glm_20s.variables['flash_lat'][:])
    lons_flash = np.append(lons_flash, glm_20s.variables['flash_lon'][:])

    # Increment the date_ini
    date_ini = str(datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))

# Stack and transpose the lat lons
values = np.vstack((lats, lons)).T

# Get the counts
points, counts = np.unique(values, axis=0, return_counts=True)

# Get the counts indices
idx = counts.argsort()

#========================================================================================================================#
#                                                 CREATE THE FIGURE
#========================================================================================================================#
# Choose the plot size (width x height, in inches)
plt.figure(figsize=(10,10))

# Use the Geostationary projection in cartopy
ax = plt.axes(projection=ccrs.PlateCarree())

# Define the image extent
ax.set_extent([extent[0], extent[2], extent[1], extent[3]], ccrs.PlateCarree())

# Define the data extent
img_extent = [extent[0], extent[2], extent[1], extent[3]]

# Plot the image
img = ax.imshow(dataset_ir['Band1'], vmin=-80, vmax=40, cmap='gray_r', origin='upper', extent=img_extent, zorder=1)

# Plot the GLM Data
glm = plt.scatter(points[idx,1], points[idx,0], vmin=0, vmax=1500, s=counts[idx]*0.1, c=counts[idx], cmap="jet", alpha=0.5, zorder=2)

# Add coastlines, borders and gridlines
ax.coastlines(resolution='10m', color='white', linewidth=0.8, zorder=3)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5, zorder=4)

gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True, zorder=5)
plt.xlim(extent[0], extent[2])
plt.ylim(extent[1], extent[3])

# Plot states
shapefile = list(shpreader.Reader('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

# Add the glm colorbar
plt.colorbar(glm, label='GLM Density', extend='max', orientation='horizontal', pad=0.10, fraction=0.05)

# Add a colorbar
plt.colorbar(img, label='Brightness Temperatures (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)

# Add a title
plt.title('GOES-16 IR + GLM Flashes', fontweight='bold', fontsize=10, loc='left')
date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))
date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(minutes=10))
plt.title(str(date_ini) + " - " + str(date_end), fontsize="10", loc="right")

# Save the image
plt.savefig(f'{output}/script_5_{yyyy}{mm}{dd}_{hh}{mn}.png', bbox_inches='tight', pad_inches=0, dpi=300)

# Show the image
plt.show()

#**Script 06** - Evolução Temporal da Temperatura de Brilho do IR e Flashes

## Mapa
- O processamento de 2h de imagens GOES-16 + GLM demora aproximadamente 5 min.

In [ ]:
#========================================================================================================================#
#                                            REQUIRED MODULES
#========================================================================================================================#
from netCDF4 import Dataset                                 # Read / Write NetCDF4 files
import matplotlib.pyplot as plt                             # Plotting library
import cartopy, cartopy.crs as ccrs                         # Plot maps
from datetime import timedelta, date, datetime              # Basic Dates and time types
import os                                                   # Miscellaneous operating system interfaces
from osgeo import gdal                                      # Python bindings for GDAL
import numpy as np                                          # Scientific computing with Python
from matplotlib import cm                                   # Colormap handling utilities
from utilities import download_CMI, download_GLM, loadCPT   # Import the CPT convert function   # Our function for download
from utilities import reproject                             # Our function for reproject
import cartopy.io.shapereader as shpreader                  # Read shapefiles
import xarray as xr                                         # Work with multidimensional arrays
import pandas as pd                                         # Work with dataframes
import salem                                                # Work with shapefiles
import geopandas as gpd                                     # Work with geodataframes
gdal.PushErrorHandler('CPLQuietErrorHandler')               # Ignore GDAL warnings
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                        INITIAL DEFINITIONS
#========================================================================================================================#
# Create the input and output directories
input = "/content/samples"; os.makedirs(input, exist_ok=True)
output = "/content/output"; os.makedirs(output, exist_ok=True)

# Channel
band = '13'

# Variable
var = 'CMI'

# Desired extent
lonmin, lonmax, latmin, latmax = -55.8, -49.0, -32.1, -28.5
extent = [lonmin, lonmax, latmin, latmax] # Min lon, Max lon, Min lat, Max lat

# Reading Santa Maria shapefile
santa_maria = salem.read_shapefile('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/SANTA_MARIA_MUN.shp')

#========================================================================================================================#
#                                              LOOP OF IMAGES
#========================================================================================================================#
# Variables of time evolution of temperature and flashes
temp_min_sm, temp_mean_sm, flash_total_sm, time_images = [], [], [], []

# Images Loop
for date_image in pd.date_range('202404301300', '202404301500', freq='10min'):

    #--------------------------------------------------------------------------#
    #                                DATE
    #--------------------------------------------------------------------------#
    # Datetime to process
    yyyymmddhhmn = date_image.strftime('%Y%m%d%H%M') # '202404301300'

    # Initial time and date
    yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
    mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
    dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
    hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
    mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')
    time_images.append(f'{hh}:{mn}')

    print('#=====================================================================================================#')
    print(f'                           PROCESSING THE IMAGE = {yyyy}-{mm}-{dd} {hh}{mn} UTC'                       )
    print('#=====================================================================================================#')

    # Download the file
    file_ir = download_CMI(yyyymmddhhmn, band, input)

    #--------------------------------------------------------------------------#
    #                       REPROJECT THE DATA
    #--------------------------------------------------------------------------#
    # Open the file
    img = gdal.Open(f'NETCDF:{input}/{file_ir}.nc:' + var)

    # Data Quality Flag (DQF)
    dqf = gdal.Open(f'NETCDF:{input}/{file_ir}.nc:DQF')

    # Read the header metadata
    metadata = img.GetMetadata()
    scale = float(metadata.get(var + '#scale_factor'))
    offset = float(metadata.get(var + '#add_offset'))
    undef = float(metadata.get(var + '#_FillValue'))
    dtime = metadata.get('NC_GLOBAL#time_coverage_start')

    # Load the data
    ds_cmi = img.ReadAsArray(0, 0, img.RasterXSize, img.RasterYSize).astype(float)
    ds_dqf = dqf.ReadAsArray(0, 0, dqf.RasterXSize, dqf.RasterYSize).astype(float)

    # Apply the scale, offset and convert to celsius
    ds_cmi = (ds_cmi * scale + offset) - 273.15

    # Apply NaN's where the quality flag is greater than 1
    ds_cmi[ds_dqf > 1] = np.nan

    # Reproject the file
    filename_ret = f'{output}/IR_{yyyymmddhhmn}.nc'
    reproject(filename_ret, img, ds_cmi, extent, undef)

    #--------------------------------------------------------------------------#
    #                       READ THE REPROJECTED FILE
    #--------------------------------------------------------------------------#
    dataset_ir = xr.open_dataset(filename_ret).sel(lon=slice(lonmin, lonmax), lat=slice(latmax, latmin))

    #--------------------------------------------------------------------------#
    #                         GET GLM DATA
    #--------------------------------------------------------------------------#
    # date the actual image and next image
    date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))
    date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(minutes=10))

    # GLM accumulation loop
    lats_flash, lons_flash = np.array([]), np.array([])
    while (date_ini <= date_end):

        # Date structure
        yyyymmddhhmnss = datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

        # Download the file
        file_glm = download_GLM(yyyymmddhhmnss, input)

        # Read the file
        glm_20s = Dataset(f'{input}/{file_glm}.nc')

        # Append lats / longs
        lats_flash = np.append(lats_flash, glm_20s.variables['flash_lat'][:])
        lons_flash = np.append(lons_flash, glm_20s.variables['flash_lon'][:])

        # Increment the date_ini
        date_ini = str(datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))
    #--------------------------------------------------------------------------#

    # Select the flashes inside the region
    # put the flashes into dataframe
    data_flash = {'lat': lats_flash, 'lon': lons_flash}
    df = pd.DataFrame(data_flash)

    # select the flashes from region of interest
    df_flash_filtered = df[ (df['lon'] > lonmin) & (df['lon'] < lonmax) & (df['lat'] > latmin) & (df['lat'] < latmax)]

    # transform from dataframe to array
    lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

    #--------------------------------------------------------------------------#
    #                       CREATE THE FIGURE
    #--------------------------------------------------------------------------#
    # Choose the plot size (width x height, in inches)
    plt.figure(figsize=(14,11))

    # Use the Geostationary projection in cartopy
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Converts a CPT file to be used in Python
    cpt = loadCPT('IR4AVHRR6.cpt')
    cmap = cm.colors.LinearSegmentedColormap('cpt', cpt)

    # Plot the image
    img = ax.imshow(dataset_ir['Band1'], origin='upper', vmin=-103.0, vmax=84, extent=[lonmin, lonmax, latmin, latmax], cmap=cmap, alpha=1.0)

    # Plot the GLM Data
    glm = plt.scatter(lons_flash_filtered, lats_flash_filtered, transform=ccrs.PlateCarree(), marker='x', s=40, color='orange', zorder=2, label=f'Flashes={str(len(lats_flash_filtered)).zfill(4)}')

    # Add gridlines
    gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

    # Plot States shapefile
    shapefile = list(shpreader.Reader('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

    # Plot Santa Maria shapefile
    shapefile = list(shpreader.Reader('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/SANTA_MARIA_MUN.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='gray', facecolor='none', linewidth=3.0, zorder=3)

    # Add a colorbar
    plt.colorbar(img, label='Brightness Temperatures (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)

    # Add legend
    ax.legend(loc='lower right', ncols=1, frameon=True)

    # Add a title
    plt.title(f'GOES-16 Band 13 (10.3 µm)\n{yyyy}-{mm}-{dd} {hh}:{mn} UTC', fontweight='bold', fontsize=10, loc='left')
    plt.title('Reg.: ' + str(extent) , fontsize=10, loc='right')

    # Save the image
    plt.savefig(f'{output}/script_6a_{yyyy}{mm}{dd}_{hh}{mn}.png', bbox_inches='tight', dpi=300)

    #--------------------------------------------------------------------------#
    #            EXTRACT THE TEMPERATURE INSIDE SANTA MARIA CITY
    #--------------------------------------------------------------------------#
    # extract values only inside Santa Maria city
    data_ir_sm = dataset_ir['Band1'].salem.roi(shape=santa_maria)

    # calculate the minimum and mean temperature
    temp_min, temp_mean = float(data_ir_sm.min(('lon', 'lat'))), float(data_ir_sm.mean(('lon', 'lat')))

    # append the variables
    temp_min_sm.append(temp_min)
    temp_mean_sm.append(temp_mean)

    #--------------------------------------------------------------------------#
    #            EXTRACT THE FLASHES INSIDE SANTA MARIA CITY
    #--------------------------------------------------------------------------#
    # read Santa Maria shapefile into GeoDataframe
    santamaria_gpd = gpd.read_file('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/SANTA_MARIA_MUN.shp')

    # new geodataframe
    df_flash_filtered_gpd = gpd.GeoDataFrame(df_flash_filtered.reset_index(), geometry=gpd.points_from_xy(df_flash_filtered.lon, df_flash_filtered.lat))

    # assign coordinate system (CRS)
    df_flash_filtered_gpd.crs = santamaria_gpd.crs

    # apply the mask
    df_flash_filtered_gpd_sm = gpd.overlay(df_flash_filtered_gpd, santamaria_gpd, how='intersection')

    # append flashes
    flash_total_sm.append(df_flash_filtered_gpd_sm.shape[0])
    print('\n')

#--------------------------------------------------------------------------#
#               INSERT INTO DATAFRAME
#--------------------------------------------------------------------------#
data_ts = {'time': time_images,
           'temp_min_sm': temp_min_sm,
           'temp_mean_sm': temp_mean_sm,
           'flash_total_sm': flash_total_sm}

df_ts = pd.DataFrame(data_ts)

In [ ]:
# flashes
df_flash_filtered

In [ ]:
# flashes for Santa Maria
df_flash_filtered_gpd_sm

In [ ]:
# number flashes before and after apply filter
print(df_flash_filtered.shape[0], df_flash_filtered_gpd_sm.shape[0])

In [ ]:
# showing the dataframe with temperature and flashes
df_ts

## Animação das imagens

In [ ]:
# import library
import imageio # Make animations
import glob    # To list files

# list the images for animation
files = sorted(glob.glob('/content/output/script_6a*png'))

# make animation
images = []
for file in files:
    images.append(imageio.imread(file))

# salve the animation
imageio.mimsave('/content/output/script_6b_animacao.gif',
                images,
                duration=700,
                loop=0)

# show the animation
print("\nAbrindo o GIF..\n")
from IPython.display import Image
Image(open(f'/content/output/script_6b_animacao.gif','rb').read(), width=600)

##Evolução temporal

In [ ]:
#==================================================================================#
#                       INITIAL CONFIGURATION
#==================================================================================#
# import library
import matplotlib.pyplot as plt

# size of figure
fig, ax1 = plt.subplots(figsize=(10, 6))

#==================================================================================#
#             PLOT-1: plotting temperature on the primary y-axis (ax1)
#==================================================================================#
# plot figure
ax1.plot(df_ts['time'].values, df_ts['temp_min_sm'].values, marker='o', color='blue', linestyle='--', label='BT Minimum')
ax1.plot(df_ts['time'].values, df_ts['temp_mean_sm'].values, marker='o', color='black', linestyle='--', label='BT Mean')

# X and Y-axis names
ax1.set_xlabel('Time (UTC)', color='black', size=15)
ax1.set_ylabel('Brigthness Temperature ($^o$C)', color='black', size=15)

# X and Y-axis configuration
ax1.tick_params(axis='x', labelcolor='black', labelsize=15, rotation=30)
ax1.tick_params(axis='y', labelcolor='black', labelsize=15)

# add legend
ax1.legend(loc='upper left', frameon=False)

#==================================================================================#
#             PLOT-2: plotting flash on the secundary y-axis (ax2)
#==================================================================================#
# add additional y-axis
ax2 = ax1.twinx()

# plot figure
ax2.plot(df_ts['time'].values, df_ts['flash_total_sm'].values, marker='o', color='green', label='Total Flash')

# Y-axis name
ax2.set_ylabel('Total Flashes (fl/10min)', color='black', size=15)

# Y-axis configuration
ax2.tick_params(axis='y', labelcolor='black', labelsize=15)

# add legend
ax2.legend(bbox_to_anchor=(0.178, 0.90), frameon=False)

# add title for figure
plt.title(f'Temporal Evolution of Temperature (ABI) and Flashes (GLM)')

#==================================================================================#
#                     SALVE AND SHOW THE FIGURE
#==================================================================================#
# save the image
plt.savefig(f'{output}/script_6c_temporal_evolution.png', bbox_inches='tight', dpi=300)

# show the figure
plt.show()